# Titanic Survival Machine Learning Model

This notebook contains a **quick** attempt at building a machine learning model for predicting the surival of a passenger on the ship [Titanic](https://en.wikipedia.org/wiki/RMS_Titanic), based on the dataset hosted at [Kaggle](https://www.kaggle.com). The ultimate aim of this work is to demonstrate an archytypal 'data science workflow' that includes some data exploration, feature engineering, model training and selection and to yield a model that can be used in other projects downstream. We have already downloaded the data from Kaggle, in CSV format, to the `data` directory in this project's root directory. 

## Package Imports and Environment Setup

In [316]:
import warnings
from datetime import datetime
from typing import Any, Callable, Dict, Iterable

import joblib
import pandas as pd
from pandas import DataFrame
from numpy import ndarray
from sklearn.base import BaseEstimator
from sklearn.compose import make_column_transformer
from sklearn.ensemble import (
    GradientBoostingClassifier, RandomForestClassifier, VotingClassifier)
from sklearn.exceptions import NotFittedError
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import (
    BaseCrossValidator, GridSearchCV, StratifiedKFold, train_test_split)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

warnings.filterwarnings('ignore')

## Load Dataset

In [319]:
data = pd.read_csv('data/train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


## Summary Statistics for Numeric Data

In [5]:
(data
 .drop(['PassengerId'], axis=1)
 .describe())

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Data Description

- `Survived` - Survival (0 = No, 1 = Yes);
- `Pclass` - Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd);
- `Sex` - sex (male/female);
- `SibSp` - # of siblings / spouses aboard the Titanic (positive integer);
- `Parch` - # of parents / children aboard the Titanic (positive integer);
- `Ticket` - Ticket number (positive integer);
- `Fare` - Passenger fare (positive real);
- `Cabin` - Cabin number (string); and,
- `Embarked` - Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton).

Note, that some children travelled only with a nanny, therefore `parch=0` for them.

In [6]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Extract Class Labels

The class labels for this classification task are in the `Survived` column of the training data. We split this column into seperate vector for model training.

In [7]:
labels = data.Survived.values

Take a look at the number of observations assigned to each class (i.e. look for the extent of the class imbalance).

In [8]:
survived_fraction = labels.sum() / labels.shape[0]
print(f'{survived_fraction:.1%} of passengers are classified as surviving')

38.4% of passengers are classified as surviving


## Spit Dataset into Train and Test Subsets

Use stratified sampling to ensure that classes are fairly represented in both train and test data.

In [69]:
train_data, test_data, y_train, y_test = train_test_split(
    data,
    labels,
    random_state=42,
    test_size=0.1,
    stratify=labels)

## Feature Engineering

We will **drop** the following columns from the final set of features:

- `PassengerId` - no apparent information content;
- `Survived` - i.e. the class labels we wish to predict;
- `Name` - no apparent predictive information content;
- `Ticket` - no apparent information content; and,
- `Cabin` - too many missing values to work with and `Pclass` and `Fare` should be adequate descriptors of cabin.

We will need to infer the missing values for the following columns (plus label each observation as having missing data):

- `Age` - using `Sex` and whether or not child or adult as inferred from `Name` and `Parch`; and,
- `Embarked` - assign to a new category labelled as `UNKNOWN`.

We will one-hot-encode the following categorical variables:

- `Pclass`;
- `Sex`; and,
- `Embarked`.

### Missing Data Imputers

In [68]:
age_imputer = SimpleImputer(strategy='mean')
embarked_imputer = SimpleImputer(strategy='constant', fill_value='UNKNOWN')

### Configure and Fit Feature Engineering Pipeline

In [64]:
feature_pipeline = make_column_transformer(
    (make_pipeline(age_imputer, StandardScaler()), ['Age']),
    (make_pipeline(embarked_imputer, OneHotEncoder()), ['Embarked']),
    (StandardScaler(), ['Fare']),
    (OneHotEncoder(), ['Pclass', 'Sex']))

feature_pipeline.fit(train_data);

### Transform Train and Test Data

In [71]:
X_train = feature_pipeline.transform(train_data)
X_test = feature_pipeline.transform(test_data)

X_train

array([[-1.04511068e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.04511068e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-2.72194763e-16,  0.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [ 1.32998578e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.55983382e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-9.68494661e-01,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00]])

## Model Selection

We are fairly certain that the final decision boundary needs to be non-linear and that model training will benefit from models that can use weighted loss function. With this in mind, we will test the following model classes:

- Support Vector Machines (SVM);
- Random Forests (RF);
- Gradient Boosting Machine (GBM); and,
- Voting Classifier based on combining the above models.

### Experimental Setup

We will use 5-fold cross validation for hyper-parameter tuning and then asses performance on the test data.

In [110]:
cross_validator = StratifiedKFold(n_splits=5, random_state=42)

### Test Metrics

We will use the following metrics to assess model performance:

- Area Under ROC Curve (AUC); and,
- accuracy.

In [166]:
metrics = {'auc': roc_auc_score,
           'accuray': accuracy_score}

### Automation

We define a simple class for automating grid-search, cross-validation and metric calculation across all model types.

In [268]:
MetricScorer = Callable[[Iterable[int], Iterable[int]], float]


class MLExperimentRunner:
    """Model training and metric calculation automation."""
    
    def __init__(self,
                 metrics: Dict[str, MetricScorer],
                 cross_validator: BaseCrossValidator,
                 model_selection_metric: str,
                 X_train: ndarray,
                 y_train: ndarray,
                 X_test: ndarray,
                 y_test: ndarray,) -> None:
        
        self.metrics = metrics
        self.cv = cross_validator
        self.selection_metric = model_selection_metric
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        return None
    
    def run(self, estimator: BaseEstimator,
            param_grid: Dict[str, Any], n_results: int = 10) -> BaseEstimator:
        """Run an experiment."""
        
        scorers = {name: make_scorer(metric)
                   for name, metric in self.metrics.items()}
        
        experiment = GridSearchCV(
            estimator, param_grid, scoring=scorers, cv=self.cv,
            refit=self.selection_metric, n_jobs=2)
        
        experiment.fit(self.X_train, self.y_train)
        
        self._display_results(experiment.cv_results_, n_results)
        self._display_test_metrics(experiment.best_estimator_)
        
        return experiment.best_estimator_

    def _display_test_metrics(self, estimator: BaseEstimator) -> None:
        """Print metrics for test data to stdout."""
        
        predictions = estimator.predict(self.X_test)
        results = {
            metric_name: metric_func(predictions, self.y_test)
            for metric_name, metric_func in self.metrics.items()}
        
        print('----------------------------------------')
        print('-- TEST DATA METRICS (best estimator) --')
        print('---------------------------------------')
        for metric_name, metric_value in results.items():
            print(f'{metric_name}: {metric_value:.4f}')
        print('----------------------------------------')
        
        return None

    def _display_results(self, cv_results: Dict[str, Any],
                         n_results: int) -> None:
        """Display cross validation results as DataFrame."""
        
        cv_data = DataFrame(cv_results)

        metric_col_names = [
            e 
            for name in self.metrics.keys()
            for e in [f'mean_test_{name}', f'std_test_{name}']]

        selection_metric_col_name = f'mean_test_{self.selection_metric}'

        cv_data.sort_values(
            by=selection_metric_col_name, ascending=False, inplace=True)
        cv_data.reset_index(inplace=True)

        display(cv_data[['params'] + metric_col_names].head(n_results))

        return None

And we then generate an instance of the experiment runner for the current task.

In [269]:
ml_experiment = MLExperimentRunner(
    metrics, cross_validator, 'auc', X_train, y_train, X_test, y_test)

### Support Vector Machine

In [280]:
svm_model = ml_experiment.run(
    SVC(class_weight='balanced', gamma='auto', probability=True),
    {'C': [0.01, 0.1, 1, 10, 100],
     'kernel': ['linear', 'rbf']})

,params,mean_test_auc,std_test_auc,mean_test_accuray,std_test_accuray
0,"{'C': 100, 'kernel': 'rbf'}",0.790843,0.030378,0.805243,0.028932
1,"{'C': 10, 'kernel': 'rbf'}",0.787953,0.039333,0.803995,0.037089
2,"{'C': 1, 'kernel': 'rbf'}",0.778869,0.032124,0.792759,0.030329
3,"{'C': 0.1, 'kernel': 'rbf'}",0.770731,0.031651,0.786517,0.030487
4,"{'C': 0.1, 'kernel': 'linear'}",0.770090,0.034818,0.790262,0.034360
5,"{'C': 1, 'kernel': 'linear'}",0.770090,0.034818,0.790262,0.034360
6,"{'C': 10, 'kernel': 'linear'}",0.770090,0.034818,0.790262,0.034360
7,"{'C': 100, 'kernel': 'linear'}",0.770090,0.034818,0.790262,0.034360
8,"{'C': 0.01, 'kernel': 'linear'}",0.769075,0.034437,0.789014,0.034277
9,"{'C': 0.01, 'kernel': 'rbf'}",0.522359,0.021938,0.594257,0.064600


----------------------------------------
-- TEST DATA METRICS (best estimator) --
---------------------------------------
auc: 0.7784
accuray: 0.7889
----------------------------------------


### Random Forest

In [273]:
rf_model = ml_experiment.run(
    RandomForestClassifier(class_weight='balanced', random_state=42),
    {'n_estimators': [10, 100, 1000],
     'min_samples_split': [2, 4, 6, 8, 10],
     'max_depth': [2, 4, 6, 8, 10, None]})

,params,mean_test_auc,std_test_auc,mean_test_accuray,std_test_accuray
0,"{'max_depth': 10, 'min_samples_split': 2, 'n_e...",0.828182,0.027416,0.845194,0.024508
1,"{'max_depth': 10, 'min_samples_split': 4, 'n_e...",0.826142,0.029978,0.842697,0.026080
2,"{'max_depth': 10, 'min_samples_split': 4, 'n_e...",0.825143,0.025284,0.841448,0.021473
3,"{'max_depth': 10, 'min_samples_split': 2, 'n_e...",0.824505,0.032608,0.841448,0.028152
4,"{'max_depth': 10, 'min_samples_split': 6, 'n_e...",0.823895,0.030275,0.841448,0.026197
5,"{'max_depth': 8, 'min_samples_split': 4, 'n_es...",0.823713,0.027507,0.838951,0.022889
6,"{'max_depth': 8, 'min_samples_split': 4, 'n_es...",0.822480,0.027492,0.838951,0.024386
7,"{'max_depth': 8, 'min_samples_split': 2, 'n_es...",0.822096,0.028564,0.837703,0.026750
8,"{'max_depth': None, 'min_samples_split': 6, 'n...",0.822086,0.028906,0.837703,0.024777
9,"{'max_depth': None, 'min_samples_split': 4, 'n...",0.822080,0.026496,0.837703,0.021716


----------------------------------------
-- TEST DATA METRICS (best estimator) --
---------------------------------------
auc: 0.8256
accuray: 0.8333
----------------------------------------


### Gradient Boosting Machine

In [275]:
gbm_model = ml_experiment.run(
    GradientBoostingClassifier(random_state=42),
    {'n_estimators': [10, 100, 1000],
     'learning_rate': [0.01, 0.1, 1, 10],
     'min_samples_split': [2, 4, 6, 8, 10],
     'max_depth': [2, 4, 8, 10, None]})

,params,mean_test_auc,std_test_auc,mean_test_accuray,std_test_accuray
0,"{'learning_rate': 0.1, 'max_depth': 4, 'min_sa...",0.822430,0.038088,0.838951,0.031344
1,"{'learning_rate': 0.1, 'max_depth': 2, 'min_sa...",0.821754,0.045520,0.838951,0.036210
2,"{'learning_rate': 0.1, 'max_depth': 2, 'min_sa...",0.820127,0.042660,0.837703,0.034516
3,"{'learning_rate': 0.1, 'max_depth': 2, 'min_sa...",0.819740,0.040412,0.836454,0.032256
4,"{'learning_rate': 0.1, 'max_depth': 2, 'min_sa...",0.819740,0.040412,0.836454,0.032256
5,"{'learning_rate': 0.1, 'max_depth': 2, 'min_sa...",0.819740,0.040412,0.836454,0.032256
6,"{'learning_rate': 0.1, 'max_depth': 4, 'min_sa...",0.819560,0.039930,0.837703,0.032984
7,"{'learning_rate': 0.1, 'max_depth': 4, 'min_sa...",0.817325,0.041290,0.836454,0.036011
8,"{'learning_rate': 0.01, 'max_depth': 4, 'min_s...",0.816318,0.031692,0.835206,0.026700
9,"{'learning_rate': 0.01, 'max_depth': 4, 'min_s...",0.816085,0.041838,0.836454,0.034097


----------------------------------------
-- TEST DATA METRICS (best estimator) --
---------------------------------------
auc: 0.8044
accuray: 0.8111
----------------------------------------


### Voting Classifier

In [283]:
voting_model = ml_experiment.run(
    VotingClassifier([
        ('svm', svm_model), ('rf', rf_model), ('gbm', gbm_model)]),
    {'voting': ['hard', 'soft']})

,params,mean_test_auc,std_test_auc,mean_test_accuray,std_test_accuray
0,{'voting': 'hard'},0.820606,0.032044,0.838951,0.026112
1,{'voting': 'soft'},0.815900,0.033738,0.837703,0.029198


----------------------------------------
-- TEST DATA METRICS (best estimator) --
---------------------------------------
auc: 0.8256
accuray: 0.8333
----------------------------------------


## Assemble Final Prediction Pipeline

We will choose the random forest model to combine with the feature pipeline and then persist to disk.

In [304]:
prediction_pipeline = make_pipeline(feature_pipeline, rf_model)

### Re-Fit Model on Entire Dataset

In [324]:
prediction_pipeline.fit(data, data.Survived)

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('pipeline-1', Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mea...tors=1000, n_jobs=None, oob_score=False,
            random_state=42, verbose=0, warm_start=False))])

### Test Pipeline

In [325]:
prediction_pipeline.predict(test_data)

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1])

### Persist Model

In [326]:
base_model_name = 'titanic-ml'
directory = 'models'
timestamp = datetime.now().isoformat(timespec='seconds')

joblib.dump(prediction_pipeline, 
            f'{directory}/{base_model_name}-{timestamp}.joblib')

['models/titanic-ml-2019-02-11T16:38:12.joblib']